# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f69acc3ae50>
├── 'a' --> tensor([[ 0.8667, -1.2557, -2.0797],
│                   [-0.3077, -0.6453,  1.2652]])
└── 'x' --> <FastTreeValue 0x7f698837a2b0>
    └── 'c' --> tensor([[ 2.1441, -0.1156,  0.0939, -1.5727],
                        [-1.1628,  0.8543,  1.1539,  0.5806],
                        [ 0.0546,  0.4014, -0.1263,  0.9575]])

In [4]:
t.a

tensor([[ 0.8667, -1.2557, -2.0797],
        [-0.3077, -0.6453,  1.2652]])

In [5]:
%timeit t.a

69.8 ns ± 0.64 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f69acc3ae50>
├── 'a' --> tensor([[ 1.4107, -0.3386, -1.4592],
│                   [-0.4701, -0.6296,  1.2823]])
└── 'x' --> <FastTreeValue 0x7f698837a2b0>
    └── 'c' --> tensor([[ 2.1441, -0.1156,  0.0939, -1.5727],
                        [-1.1628,  0.8543,  1.1539,  0.5806],
                        [ 0.0546,  0.4014, -0.1263,  0.9575]])

In [7]:
%timeit t.a = new_value

79 ns ± 0.622 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8667, -1.2557, -2.0797],
               [-0.3077, -0.6453,  1.2652]]),
    x: Batch(
           c: tensor([[ 2.1441, -0.1156,  0.0939, -1.5727],
                      [-1.1628,  0.8543,  1.1539,  0.5806],
                      [ 0.0546,  0.4014, -0.1263,  0.9575]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8667, -1.2557, -2.0797],
        [-0.3077, -0.6453,  1.2652]])

In [11]:
%timeit b.a

73.2 ns ± 1.63 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2108,  0.5938,  1.1798],
               [ 0.5847, -0.0619,  2.5925]]),
    x: Batch(
           c: tensor([[ 2.1441, -0.1156,  0.0939, -1.5727],
                      [-1.1628,  0.8543,  1.1539,  0.5806],
                      [ 0.0546,  0.4014, -0.1263,  0.9575]]),
       ),
)

In [13]:
%timeit b.a = new_value

568 ns ± 8.92 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

915 ns ± 7.24 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.4 µs ± 224 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

272 µs ± 9.82 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

256 µs ± 6.29 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f68f4ef4790>
├── 'a' --> tensor([[[ 0.8667, -1.2557, -2.0797],
│                    [-0.3077, -0.6453,  1.2652]],
│           
│                   [[ 0.8667, -1.2557, -2.0797],
│                    [-0.3077, -0.6453,  1.2652]],
│           
│                   [[ 0.8667, -1.2557, -2.0797],
│                    [-0.3077, -0.6453,  1.2652]],
│           
│                   [[ 0.8667, -1.2557, -2.0797],
│                    [-0.3077, -0.6453,  1.2652]],
│           
│                   [[ 0.8667, -1.2557, -2.0797],
│                    [-0.3077, -0.6453,  1.2652]],
│           
│                   [[ 0.8667, -1.2557, -2.0797],
│                    [-0.3077, -0.6453,  1.2652]],
│           
│                   [[ 0.8667, -1.2557, -2.0797],
│                    [-0.3077, -0.6453,  1.2652]],
│           
│                   [[ 0.8667, -1.2557, -2.0797],
│                    [-0.3077, -0.6453,  1.2652]]])
└── 'x' --> <FastTreeValue 0x7f69880bfe80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

43.2 µs ± 631 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f68f4f6c490>
├── 'a' --> tensor([[ 0.8667, -1.2557, -2.0797],
│                   [-0.3077, -0.6453,  1.2652],
│                   [ 0.8667, -1.2557, -2.0797],
│                   [-0.3077, -0.6453,  1.2652],
│                   [ 0.8667, -1.2557, -2.0797],
│                   [-0.3077, -0.6453,  1.2652],
│                   [ 0.8667, -1.2557, -2.0797],
│                   [-0.3077, -0.6453,  1.2652],
│                   [ 0.8667, -1.2557, -2.0797],
│                   [-0.3077, -0.6453,  1.2652],
│                   [ 0.8667, -1.2557, -2.0797],
│                   [-0.3077, -0.6453,  1.2652],
│                   [ 0.8667, -1.2557, -2.0797],
│                   [-0.3077, -0.6453,  1.2652],
│                   [ 0.8667, -1.2557, -2.0797],
│                   [-0.3077, -0.6453,  1.2652]])
└── 'x' --> <FastTreeValue 0x7f68f4e86f40>
    └── 'c' --> tensor([[ 2.1441, -0.1156,  0.0939, -1.5727],
                        [-1.1628,  0.8543,  1.1539,  0.5806],
                 

In [23]:
%timeit t_cat(trees)

38.2 µs ± 502 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

73.3 µs ± 1.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.8667, -1.2557, -2.0797],
                [-0.3077, -0.6453,  1.2652]],
       
               [[ 0.8667, -1.2557, -2.0797],
                [-0.3077, -0.6453,  1.2652]],
       
               [[ 0.8667, -1.2557, -2.0797],
                [-0.3077, -0.6453,  1.2652]],
       
               [[ 0.8667, -1.2557, -2.0797],
                [-0.3077, -0.6453,  1.2652]],
       
               [[ 0.8667, -1.2557, -2.0797],
                [-0.3077, -0.6453,  1.2652]],
       
               [[ 0.8667, -1.2557, -2.0797],
                [-0.3077, -0.6453,  1.2652]],
       
               [[ 0.8667, -1.2557, -2.0797],
                [-0.3077, -0.6453,  1.2652]],
       
               [[ 0.8667, -1.2557, -2.0797],
                [-0.3077, -0.6453,  1.2652]]]),
    x: Batch(
           c: tensor([[[ 2.1441, -0.1156,  0.0939, -1.5727],
                       [-1.1628,  0.8543,  1.1539,  0.5806],
                       [ 0.0546,  0.4014, -0.1263,  0.9575]],
         

In [26]:
%timeit Batch.stack(batches)

96.8 µs ± 914 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.8667, -1.2557, -2.0797],
               [-0.3077, -0.6453,  1.2652],
               [ 0.8667, -1.2557, -2.0797],
               [-0.3077, -0.6453,  1.2652],
               [ 0.8667, -1.2557, -2.0797],
               [-0.3077, -0.6453,  1.2652],
               [ 0.8667, -1.2557, -2.0797],
               [-0.3077, -0.6453,  1.2652],
               [ 0.8667, -1.2557, -2.0797],
               [-0.3077, -0.6453,  1.2652],
               [ 0.8667, -1.2557, -2.0797],
               [-0.3077, -0.6453,  1.2652],
               [ 0.8667, -1.2557, -2.0797],
               [-0.3077, -0.6453,  1.2652],
               [ 0.8667, -1.2557, -2.0797],
               [-0.3077, -0.6453,  1.2652]]),
    x: Batch(
           c: tensor([[ 2.1441, -0.1156,  0.0939, -1.5727],
                      [-1.1628,  0.8543,  1.1539,  0.5806],
                      [ 0.0546,  0.4014, -0.1263,  0.9575],
                      [ 2.1441, -0.1156,  0.0939, -1.5727],
                      [-1.1628,  

In [28]:
%timeit Batch.cat(batches)

171 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

549 µs ± 14.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
